In [1]:
import pandas as pd
import numpy as np
from scipy.stats import norm

def cumulative_sums_test(sequence: str, mode: int):
    """
    Executa o Cumulative Sums (Cusum) Test do NIST SP800-22.

    Parâmetros:
    - sequence (str): sequência binária (ex: "101001...")
    - mode (int): 0 para forward, 1 para backward

    Retorna:
    - z (float): maior excursão da caminhada aleatória
    - p_value (float): valor-p do teste
    """
    n = len(sequence)

    # Etapa 1: converter bits para -1 e +1
    x = np.array([2 * int(bit) - 1 for bit in sequence], dtype=int)

    # Etapa 2: calcular a soma cumulativa (modo direto ou reverso)
    if mode == 0:
        s = np.cumsum(x)
    else:
        s = np.cumsum(x[::-1])

    # Etapa 3: maior excursão absoluta
    z = np.max(np.abs(s))

    # Etapa 4: cálculo do valor-p com base na distribuição normal padrão
    terms = []
    start = int((-n / z + 1) / 4)
    end = int((n / z - 1) / 4) + 1
    for k in range(start, end + 1):
        term = norm.cdf((4 * k + 1) * z / np.sqrt(n)) - norm.cdf((4 * k - 1) * z / np.sqrt(n))
        terms.append(term)

    sum1 = np.sum(terms)

    terms = []
    start = int((-n / z - 3) / 4)
    end = int((n / z - 1) / 4) + 1
    for k in range(start, end + 1):
        term = norm.cdf((4 * k + 3) * z / np.sqrt(n)) - norm.cdf((4 * k + 1) * z / np.sqrt(n))
        terms.append(term)

    sum2 = np.sum(terms)

    p_value = 1.0 - sum1 + sum2

    return z, p_value

# Caminho do arquivo
file_path = "4-seqbinaria_sorteios.csv"
output_path = "resultado_teste_somas_cumulativas_sorteios.csv"

try:
    df = pd.read_csv(file_path, sep="\t", header=None, names=['Candidato', 'Mediana', 'Sequencia'])

    results = []

    for _, row in df.iterrows():
        seq_str = row['Sequencia']
        seq_len = len(seq_str)
        seq_binaria = np.array(list(seq_str), dtype=int)
        n_1s = np.sum(seq_binaria)
        n_0s = seq_len - n_1s

        z_f, p_f = cumulative_sums_test(seq_str, 0)
        z_b, p_b = cumulative_sums_test(seq_str, 1)
        p_min = min(p_f, p_b)

        # Classificação com base no p-valor mínimo
        if p_min >= 0.01:
            interpretacao = "Alta aleatoriedade"
        elif 0.001 <= p_min < 0.01:
            interpretacao = "Aleatoriedade moderada"
        else:
            interpretacao = "Baixa aleatoriedade"

        results.append({
            'candidato': row['Candidato'],
            'mediana': row['Mediana'],
            'tamanho_sequencia': seq_len,
            'n_1s': n_1s,
            'n_0s': n_0s,
            'p_valor_forward': round(p_f, 6),
            'p_valor_reverse': round(p_b, 6),
            'sequencia': seq_str,
            'interpretacao': interpretacao
        })

    df_results = pd.DataFrame(results)

    # Resumo
    resumo = df_results['interpretacao'].value_counts().reset_index()
    resumo.columns = ['Interpretação', 'Quantidade']
    resumo['Faixa de p-valor'] = resumo['Interpretação'].map({
        'Alta aleatoriedade': 'p ≥ 0,01',
        'Aleatoriedade moderada': '0,001 ≤ p < 0,01',
        'Baixa aleatoriedade': 'p < 0,001'
    })
    resumo['Percentual (%)'] = (resumo['Quantidade'] / len(df_results) * 100).round(2)

    # Ordenação e exibição
    ordem = ['Alta aleatoriedade', 'Aleatoriedade moderada', 'Baixa aleatoriedade']
    resumo['ordem'] = resumo['Interpretação'].apply(lambda x: ordem.index(x))
    resumo = resumo.sort_values('ordem').drop(columns='ordem')
    resumo = resumo[['Interpretação', 'Faixa de p-valor', 'Quantidade', 'Percentual (%)']]

    print(resumo.to_string(index=False))

    # Salvar resultados completos
    df_results.to_csv(output_path, index=False)
    print()
    print()
    print(f"Arquivo '{output_path}' salvo com sucesso.")

except FileNotFoundError:
    print(f"Erro: o arquivo '{file_path}' não foi encontrado.")

except pd.errors.EmptyDataError:
    print(f"Erro: o arquivo '{file_path}' está vazio ou mal formatado.")

except Exception as e:
    print(f"Ocorreu um erro ao processar os dados: {e}")


     Interpretação Faixa de p-valor  Quantidade  Percentual (%)
Alta aleatoriedade         p ≥ 0,01         544           100.0


Arquivo 'resultado_teste_somas_cumulativas_sorteios.csv' salvo com sucesso.
